In [ ]:
# 用决策树方法对titanic生还机会进行预测
#titanic乘客数据查验
import pandas as pd
#用pandas的read_csv模块直接从互联网读取数据
titanic = pd.read_csv('http://biostat.mc.vanderbilt.edu/wiki/pub/Main/DataSets/titanic.txt')
#观察前几行数据，可以发现数据种类各异，甚至有缺失数据
titanic.head()

In [2]:
#使用pandas,数据都转为pandas独有的dataframe格式（二维数据表格），直接使用info()查看数据统计特性
titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1313 entries, 0 to 1312
Data columns (total 11 columns):
row.names    1313 non-null int64
pclass       1313 non-null object
survived     1313 non-null int64
name         1313 non-null object
age          633 non-null float64
embarked     821 non-null object
home.dest    754 non-null object
room         77 non-null object
ticket       69 non-null object
boat         347 non-null object
sex          1313 non-null object
dtypes: float64(1), int64(2), object(8)
memory usage: 112.9+ KB


In [3]:
#非常重要的一步：选择特征，我们可以了解，sex,age,pclass与能否生还起决定性因素
x=titanic[['pclass','age','sex']]
y=titanic[['survived']]

#对当前选择的特征进行查看
x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1313 entries, 0 to 1312
Data columns (total 3 columns):
pclass    1313 non-null object
age       633 non-null float64
sex       1313 non-null object
dtypes: float64(1), object(2)
memory usage: 30.8+ KB


In [4]:
#对数据进行预处理：（1）age有缺省的补全，用平均数或中位数都对模型偏离造成较小影响
#(2)sex与plass数据都是类别型的，需要转化为数值特征，用0/1代替
x['age'].fillna(x['age'].mean(), inplace=True)
x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1313 entries, 0 to 1312
Data columns (total 3 columns):
pclass    1313 non-null object
age       1313 non-null float64
sex       1313 non-null object
dtypes: float64(1), object(2)
memory usage: 30.8+ KB


d:\ProgramData\Anaconda2\lib\site-packages\pandas\core\generic.py:3660: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [5]:
#数据分割成train，与test两部分
from sklearn.cross_validation import train_test_split
x_train, x_test, y_train, y_test=train_test_split(x,y, test_size=0.25,random_state=33)

#使用scikit-learn.feature_extraction中的类型转换器
from sklearn.feature_extraction import DictVectorizer
vec=DictVectorizer(sparse=False)

#特征转换后，我们发现凡是类别型数据都单独剥离出来，独立一列特征，数值型则保持不变
x_train=vec.fit_transform(x_train.to_dict(orient='record'))
print vec.feature_names_

['age', 'pclass=1st', 'pclass=2nd', 'pclass=3rd', 'sex=female', 'sex=male']


d:\ProgramData\Anaconda2\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [6]:
#同样对测试数据进行转换
x_test=vec.transform(x_test.to_dict(orient='record'))

from sklearn.tree import DecisionTreeClassifier
#初始化模型
dtc=DecisionTreeClassifier()
#对模型进行训练
dtc.fit(x_train,y_train)
#用训练好的模型对测试数据进行预测
y_predict=dtc.predict(x_test)


In [7]:
#对乘客生还预测性能
from sklearn.metrics import classification_report
#输出预测准确性
print dtc.score(x_test, y_test)
#输出其他性能指标(精确率，召回率，F1指标等)
print classification_report(y_predict, y_test, target_names=['died','survived'])

0.781155015198
             precision    recall  f1-score   support

       died       0.91      0.78      0.84       236
   survived       0.58      0.80      0.67        93

avg / total       0.81      0.78      0.79       329

